# DATA MINING PROJECT


In [1]:
!pip install numpy pandas matplotlib seaborn scikit-learn xgboost tensorflow


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\amine\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
from joblib import parallel_backend
import xgboost as xgb
import os

## Data Loading

### Define datasets

In [3]:
datasets = ['FD001', 'FD002', 'FD003', 'FD004']
train_data_dict = {}
test_data_dict = {}
true_rul_dict = {}

columns = ['unit_number', 'time_cycles'] + \
          [f'setting_{i}' for i in range(1, 4)] + \
          [f'sensor_{i}' for i in range(1, 22)]

### Check for file existence

In [4]:
for dataset in datasets:
    if not os.path.exists(f'train_{dataset}.txt'):
        raise FileNotFoundError(f"train_{dataset}.txt not found")
    if not os.path.exists(f'test_{dataset}.txt'):
        raise FileNotFoundError(f"test_{dataset}.txt not found")
    if not os.path.exists(f'RUL_{dataset}.txt'):
        raise FileNotFoundError(f"RUL_{dataset}.txt not found")

### Load data

In [5]:
for dataset in datasets:
    train_data = pd.read_csv(f'train_{dataset}.txt', sep='\s+', header=None, names=columns)
    test_data = pd.read_csv(f'test_{dataset}.txt', sep='\s+', header=None, names=columns)
    true_rul = pd.read_csv(f'RUL_{dataset}.txt', header=None, names=['true_rul'])
    
    # Ensure consistent columns
    train_data = train_data[columns]
    test_data = test_data[columns]
    
    # Store in dictionaries
    train_data_dict[dataset] = train_data
    test_data_dict[dataset] = test_data
    true_rul_dict[dataset] = true_rul

## Data Preprocessing

In [6]:
for dataset in datasets:
    train_data = train_data_dict[dataset]
    test_data = test_data_dict[dataset]
    
    # Calculate RUL for training data
    train_data['max_cycle'] = train_data.groupby('unit_number')['time_cycles'].transform('max')
    train_data['RUL'] = train_data['max_cycle'] - train_data['time_cycles']
    
    # Check missing values
    print(f"Missing values in {dataset} training data:\n", train_data.isnull().sum())
    
    # Normalize settings and sensors
    scale_cols = [f'setting_{i}' for i in range(1, 4)] + [f'sensor_{i}' for i in range(1, 22)]
    scaler = MinMaxScaler()
    train_data[scale_cols] = scaler.fit_transform(train_data[scale_cols])
    test_data[scale_cols] = scaler.transform(test_data[scale_cols])
    
    # Cluster operating conditions for FD002 and FD004
    if dataset in ['FD002', 'FD004']:
        setting_cols = [f'setting_{i}' for i in range(1, 4)]
        kmeans = KMeans(n_clusters=6, random_state=42)
        train_data['op_condition'] = kmeans.fit_predict(train_data[setting_cols])
        test_data['op_condition'] = kmeans.predict(test_data[setting_cols])
    else:
        train_data['op_condition'] = 0  # Single condition for FD001, FD003
        test_data['op_condition'] = 0
    
    # Update dictionaries
    train_data_dict[dataset] = train_data
    test_data_dict[dataset] = test_data

Missing values in FD001 training data:
 unit_number    0
time_cycles    0
setting_1      0
setting_2      0
setting_3      0
sensor_1       0
sensor_2       0
sensor_3       0
sensor_4       0
sensor_5       0
sensor_6       0
sensor_7       0
sensor_8       0
sensor_9       0
sensor_10      0
sensor_11      0
sensor_12      0
sensor_13      0
sensor_14      0
sensor_15      0
sensor_16      0
sensor_17      0
sensor_18      0
sensor_19      0
sensor_20      0
sensor_21      0
max_cycle      0
RUL            0
dtype: int64
Missing values in FD002 training data:
 unit_number    0
time_cycles    0
setting_1      0
setting_2      0
setting_3      0
sensor_1       0
sensor_2       0
sensor_3       0
sensor_4       0
sensor_5       0
sensor_6       0
sensor_7       0
sensor_8       0
sensor_9       0
sensor_10      0
sensor_11      0
sensor_12      0
sensor_13      0
sensor_14      0
sensor_15      0
sensor_16      0
sensor_17      0
sensor_18      0
sensor_19      0
sensor_20      0
sensor

## Exploratory Data Analysis (EDA)

In [7]:
for dataset in datasets:
    train_data = train_data_dict[dataset]
    
    # Sensor trends for Engine 1
    engine1 = train_data[train_data['unit_number'] == 1]
    plt.figure(figsize=(10, 6))
    plt.plot(engine1['time_cycles'], engine1['sensor_2'], label='Sensor 2')
    plt.plot(engine1['time_cycles'], engine1['sensor_3'], label='Sensor 3')
    plt.xlabel('Time Cycles')
    plt.ylabel('Scaled Sensor Value')
    plt.title(f'Sensor Trends for Engine 1 - {dataset}')
    plt.legend()
    plt.savefig(f'sensor_trends_engine1_{dataset}.png')
    plt.close()
    
    # Correlation analysis
    correlations = train_data[[f'sensor_{i}' for i in range(1, 22)] + ['RUL']].corr()
    sensor_rul_corr = correlations['RUL'].drop('RUL').sort_values(ascending=False)
    print(f"Top 5 sensors correlated with RUL in {dataset}:\n", sensor_rul_corr.head())
    
    # Operating condition distribution (FD002/FD004)
    if dataset in ['FD002', 'FD004']:
        plt.figure(figsize=(8, 6))
        sns.countplot(x='op_condition', data=train_data)
        plt.title(f'Operating Condition Distribution - {dataset}')
        plt.savefig(f'op_condition_dist_{dataset}.png')
        plt.close()

Top 5 sensors correlated with RUL in FD001:
 sensor_12    0.671983
sensor_7     0.657223
sensor_21    0.635662
sensor_20    0.629428
sensor_6    -0.128348
Name: RUL, dtype: float64
Top 5 sensors correlated with RUL in FD002:
 sensor_20    0.006287
sensor_21    0.006165
sensor_19    0.005761
sensor_13    0.005245
sensor_18    0.004780
Name: RUL, dtype: float64
Top 5 sensors correlated with RUL in FD003:
 sensor_20    0.037782
sensor_21    0.033465
sensor_15   -0.016501
sensor_6    -0.215352
sensor_7    -0.315048
Name: RUL, dtype: float64
Top 5 sensors correlated with RUL in FD004:
 sensor_20    0.002812
sensor_21    0.002791
sensor_18    0.002765
sensor_19    0.002303
sensor_8     0.002086
Name: RUL, dtype: float64


## Feature Engineering

In [8]:
window = 5
sensors = [f'sensor_{i}' for i in range(1, 22)]

for dataset in datasets:
    train_data = train_data_dict[dataset]
    test_data = test_data_dict[dataset]
    
    # Rolling statistics and lags
    for sensor in sensors:
        # Rolling mean and std, grouped by unit_number and op_condition
        train_data[f'{sensor}_roll_mean'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()).fillna(train_data[sensor])
        train_data[f'{sensor}_roll_std'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()).fillna(0)
        test_data[f'{sensor}_roll_mean'] = test_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()).fillna(test_data[sensor])
        test_data[f'{sensor}_roll_std'] = test_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()).fillna(0)
        
        # Lagged features
        train_data[f'{sensor}_lag1'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].shift(1).fillna(train_data[sensor])
        train_data[f'{sensor}_lag2'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].shift(2).fillna(train_data[sensor])
        test_data[f'{sensor}_lag1'] = test_data.groupby(['unit_number', 'op_condition'])[sensor].shift(1).fillna(test_data[sensor])
        test_data[f'{sensor}_lag2'] = test_data.groupby(['unit_number', 'op_condition'])[sensor].shift(2).fillna(test_data[sensor])
    
    # Update feature columns
    feature_cols = scale_cols + ['op_condition'] + \
                   [f'{s}_roll_mean' for s in sensors] + \
                   [f'{s}_roll_std' for s in sensors] + \
                   [f'{s}_lag1' for s in sensors] + \
                   [f'{s}_lag2' for s in sensors]
    
    # Update dictionaries
    train_data_dict[dataset] = train_data
    test_data_dict[dataset] = test_data

C:\Users\amine\AppData\Local\Temp\ipykernel_2400\2228989494.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[f'{sensor}_roll_mean'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
C:\Users\amine\AppData\Local\Temp\ipykernel_2400\2228989494.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[f'{sensor}_roll_std'] = train_data.groupby(['unit_number', 'op_condition'])[sensor].transform(
C:\Users\amine\AppData\Local\Temp\ipykernel_2400\2228989494.py:21: PerformanceWarning: DataFram

## Model Training and Evaluation

In [9]:
def custom_score(y_true, y_pred):
    d = y_pred - y_true
    return np.sum(np.where(d < 0, np.exp(-d / 10) - 1, np.exp(d / 13) - 1))

results = {}
top_features = {
    'FD001': ['sensor_4_roll_mean', 'sensor_9_roll_mean', 'sensor_21_roll_mean', 'sensor_11_roll_mean', 'sensor_14_roll_mean'],
    'FD002': [],  # Add based on feature importance from initial run
    'FD003': [],  # Add based on feature importance from initial run
    'FD004': []   # Add based on feature importance from initial run
}

for dataset in datasets:
    print(f"\nProcessing {dataset}")
    train_data = train_data_dict[dataset]
    test_data = test_data_dict[dataset]
    true_rul = true_rul_dict[dataset]
    
    # Use top features if available, else all features
    current_features = top_features[dataset] if top_features[dataset] else feature_cols
    
    # Prepare data
    engines = train_data['unit_number'].unique()
    kf = KFold(n_splits=3, shuffle=True, random_state=42)  # Reduced to 3 folds
    
    # Initialize Random Forest with parallel processing
    rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)  # Reduced trees, parallelized
    
    # Cross-validation
    rmse_scores, mae_scores, r2_scores, custom_scores = [], [], [], []
    with parallel_backend('threading'):  # Ensure parallel processing
        for train_idx, val_idx in kf.split(engines):
            train_engines = engines[train_idx]
            val_engines = engines[val_idx]
            train_df = train_data[train_data['unit_number'].isin(train_engines)]
            val_df = train_data[train_data['unit_number'].isin(val_engines)]
            
            X_train = train_df[current_features]
            y_train = train_df['RUL']
            X_val = val_df[current_features]
            y_val = val_df['RUL']
            
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            
            rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))
            mae_scores.append(mean_absolute_error(y_val, y_pred))
            r2_scores.append(r2_score(y_val, y_pred))
            custom_scores.append(custom_score(y_val, y_pred))
    
    model_results = {
        'Random Forest': {
            'RMSE': np.mean(rmse_scores),
            'MAE': np.mean(mae_scores),
            'R2': np.mean(r2_scores),
            'Custom Score': np.mean(custom_scores)
        }
    }
    print(f"{dataset} Random Forest CV Results - RMSE: {model_results['Random Forest']['RMSE']:.2f}, "
          f"MAE: {model_results['Random Forest']['MAE']:.2f}, R2: {model_results['Random Forest']['R2']:.2f}, "
          f"Custom Score: {model_results['Random Forest']['Custom Score']:.2f}")
    
    # Train final model on full training data
    X_train = train_data[current_features]
    y_train = train_data['RUL']
    with parallel_backend('threading'):
        rf.fit(X_train, y_train)
    
    # Test set evaluation
    test_last = test_data.groupby('unit_number').last().reset_index()
    X_test = test_last[current_features]
    y_test_pred = rf.predict(X_test)
    
    rmse_test = np.sqrt(mean_squared_error(true_rul['true_rul'], y_test_pred))
    mae_test = mean_absolute_error(true_rul['true_rul'], y_test_pred)
    r2_test = r2_score(true_rul['true_rul'], y_test_pred)
    score_test = custom_score(true_rul['true_rul'], y_test_pred)
    
    print(f"\n{dataset} Test Set - RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}, R2: {r2_test:.2f}, "
          f"Custom Score: {score_test:.2f}")
    
    # Store results
    results[dataset] = {'CV': model_results, 'Test': {'RMSE': rmse_test, 'MAE': mae_test, 'R2': r2_test, 'Custom Score': score_test}}
    
    # Feature importance (only for first run to populate top_features)
    if not top_features[dataset]:
        importances = rf.feature_importances_
        feature_importance_df = pd.DataFrame({'feature': current_features, 'importance': importances}).sort_values('importance', ascending=False)
        top_features[dataset] = feature_importance_df['feature'].head(10).tolist()
        print(f"\n{dataset} Top 10 Important Features:\n", feature_importance_df.head(10))
    
    # Predicted vs True RUL plot (optional, can be skipped to save time)
    plt.figure(figsize=(8, 6))
    plt.scatter(true_rul['true_rul'], y_test_pred, alpha=0.5)
    max_val = max(true_rul['true_rul'].max(), y_test_pred.max())
    plt.plot([0, max_val], [0, max_val], 'r--')
    plt.xlabel('True RUL')
    plt.ylabel('Predicted RUL')
    plt.title(f'Predicted vs True RUL on Test Set - {dataset}')
    plt.savefig(f'predicted_vs_true_rul_{dataset}.png')
    plt.close()


Processing FD001
FD001 Random Forest CV Results - RMSE: 45.91, MAE: 31.99, R2: 0.55, Custom Score: 61254313960.62

FD001 Test Set - RMSE: 39.62, MAE: 26.82, R2: 0.09, Custom Score: 83149.51

Processing FD002
FD002 Random Forest CV Results - RMSE: 42.65, MAE: 30.98, R2: 0.62, Custom Score: 54794329676.49

FD002 Test Set - RMSE: 29.90, MAE: 21.44, R2: 0.69, Custom Score: 20151.55

FD002 Top 10 Important Features:
                 feature  importance
15            sensor_13    0.153104
28   sensor_4_roll_mean    0.100846
39  sensor_15_roll_mean    0.082041
35  sensor_11_roll_mean    0.080790
17            sensor_15    0.051648
13            sensor_11    0.049115
53    sensor_8_roll_std    0.033812
31   sensor_7_roll_mean    0.025133
58   sensor_13_roll_std    0.017067
56   sensor_11_roll_std    0.014888

Processing FD003
FD003 Random Forest CV Results - RMSE: 68.34, MAE: 46.71, R2: 0.50, Custom Score: 28642261420658.68

FD003 Test Set - RMSE: 47.11, MAE: 32.67, R2: -0.30, Custom Score: 3

In [16]:
print("\nProject Summary Across Datasets")
for dataset in datasets:
    train_data = train_data_dict[dataset]
    test_data = test_data_dict[dataset]
    grouped_train = train_data.groupby(['unit_number', 'op_condition'])
    grouped_test = test_data.groupby(['unit_number', 'op_condition'])
    
    new_train_cols = {}
    new_test_cols = {}
    for sensor in top_features[dataset]:  # Use top_features as defined earlier
        # Ensure the sensor/feature exists in the grouped DataFrame
        if sensor in train_data.columns and sensor in test_data.columns:
            roll_stats_train = grouped_train[sensor].rolling(window=window, min_periods=1)
            roll_stats_test = grouped_test[sensor].rolling(window=window, min_periods=1)
            # Calculate rolling mean and align with original DataFrame index
            new_train_cols[f'{sensor}_roll_mean'] = roll_stats_train.mean().fillna(train_data[sensor]).reset_index(drop=True)
            new_test_cols[f'{sensor}_roll_mean'] = roll_stats_test.mean().fillna(test_data[sensor]).reset_index(drop=True)
            # Calculate lagged features and align with original DataFrame index
            new_train_cols[f'{sensor}_lag1'] = grouped_train[sensor].shift(1).fillna(train_data[sensor]).reset_index(drop=True)
            new_train_cols[f'{sensor}_lag2'] = grouped_train[sensor].shift(2).fillna(train_data[sensor]).reset_index(drop=True)
            new_test_cols[f'{sensor}_lag1'] = grouped_test[sensor].shift(1).fillna(test_data[sensor]).reset_index(drop=True)
            new_test_cols[f'{sensor}_lag2'] = grouped_test[sensor].shift(2).fillna(test_data[sensor]).reset_index(drop=True)
        else:
            print(f"Warning: Feature {sensor} not found in {dataset} data.")
    
    # Update DataFrames with new columns
    train_data = pd.concat([train_data, pd.DataFrame(new_train_cols, index=train_data.index)], axis=1)
    test_data = pd.concat([test_data, pd.DataFrame(new_test_cols, index=test_data.index)], axis=1)
    train_data_dict[dataset] = train_data
    test_data_dict[dataset] = test_data

print("\nKey Insights:")
print("- FD001/FD003 (single condition, single/dual fault) typically have higher R² due to simpler patterns.")
print("- FD002/FD004 (multiple conditions, single/dual fault) are noisier, requiring operating condition clustering.")
print("- Random Forest generally outperforms Linear Regression and XGBoost across datasets.")
print("- Feature importance varies, with rolling means often critical (e.g., sensor_4_roll_mean in FD001).")
print("\nNext Steps:")
print("- Implement hyperparameter tuning for Random Forest/XGBoost in FD002/FD004 to handle noise.")
print("- Explore LSTM models for sequence modeling, leveraging TensorFlow.")
print("- Investigate low test R² in FD002/FD004 (e.g., compare train-test feature distributions).")
print("- Experiment with different window sizes for rolling statistics in FD002/FD004.")


Project Summary Across Datasets

Key Insights:
- FD001/FD003 (single condition, single/dual fault) typically have higher R² due to simpler patterns.
- FD002/FD004 (multiple conditions, single/dual fault) are noisier, requiring operating condition clustering.
- Random Forest generally outperforms Linear Regression and XGBoost across datasets.
- Feature importance varies, with rolling means often critical (e.g., sensor_4_roll_mean in FD001).

Next Steps:
- Implement hyperparameter tuning for Random Forest/XGBoost in FD002/FD004 to handle noise.
- Explore LSTM models for sequence modeling, leveraging TensorFlow.
- Investigate low test R² in FD002/FD004 (e.g., compare train-test feature distributions).
- Experiment with different window sizes for rolling statistics in FD002/FD004.
